# Homework №3

This homework will be dedicated to Keyword Spotting (KWS), streaming and speedup NN.

# Setup dependencies



In [1]:
!pip install torch

In [2]:
!pip install torchaudio==0.6.0

In [3]:
!pip install wandb

In [4]:
import torch
import torchaudio
import pandas as pd
# import multiprocessing as mp
from torch import nn
from matplotlib import pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split
from google.colab import files

# %pylab inline
# from itertools import islice
# from scipy.signal import hann
# from scipy.io import wavfile


In [5]:
import wandb
!wandb login 32f7df3c332802b239b3a621db53985fe17d319e
wandb.init(project="kws-implementation")

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: Currently logged in as: maxthehuman (use `wandb login --relogin` to force relogin)


# Config

In [6]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(7)
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

batch_size = 32
epochs = 200
n_mels = 40

freq_mask_param = 15
time_mask_param = 10

cuda


# Data

In [7]:
import os
datadir = "speech_commands"

!wget http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz -O speech_commands_v0.01.tar.gz
# alternative url: https://www.dropbox.com/s/j95n278g48bcbta/speech_commands_v0.01.tar.gz?dl=1
!mkdir {datadir} && tar -C {datadir} -xvzf speech_commands_v0.01.tar.gz 1> log

samples_by_target = {
    cls: [os.path.join(datadir, cls, name) for name in os.listdir("./speech_commands/{}".format(cls))]
    for cls in os.listdir(datadir)
    if os.path.isdir(os.path.join(datadir, cls))
}
print('Classes:', ', '.join(sorted(samples_by_target.keys())[1:]))

--2020-11-02 18:15:03--  http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.20.128, 2607:f8b0:400e:c09::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.20.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1489096277 (1.4G) [application/gzip]
Saving to: ‘speech_commands_v0.01.tar.gz’

speech_commands_v0. 100%[===================>]   1.39G   199MB/s    in 7.6s    

2020-11-02 18:15:10 (188 MB/s) - ‘speech_commands_v0.01.tar.gz’ saved [1489096277/1489096277]

mkdir: cannot create directory ‘speech_commands’: File exists
Classes: bed, bird, cat, dog, down, eight, five, four, go, happy, house, left, marvin, nine, no, off, on, one, right, seven, sheila, six, stop, three, tree, two, up, wow, yes, zero


    Choose from 1 to 3 keywords to your liking, and use the rest as negative examples.
    We recommend to use sheila and/or marvin.

###Let's use "marvin"

In [8]:
for key in samples_by_target.keys():
  print(key, len(samples_by_target[key]))

yes 2377
cat 1733
six 2369
happy 1742
seven 2377
left 2353
no 2375
stop 2380
right 2367
wow 1745
_background_noise_ 7
zero 2376
marvin 1746
dog 1746
two 2373
nine 2364
tree 1733
sheila 1734
three 2356
eight 2352
house 1750
on 2367
bed 1713
one 2370
five 2357
four 2372
up 2375
go 2372
down 2359
bird 1731
off 2357


In [9]:
len(samples_by_target)

31

In [10]:
len(samples_by_target['marvin'])

1746

### To balance marvin and non-marvin class, knowing that there are 29 other keywords, we'll take first len(marvin) / 29 tracks for each non-marvin keyword

In [11]:
COLUMN_NAMES=['wav_filename', 'class']
marvin_df = pd.DataFrame(columns=COLUMN_NAMES)
non_marvin_df = pd.DataFrame(columns=COLUMN_NAMES)

In [12]:
i = 0
for filename in samples_by_target['marvin']:
  marvin_df.loc[i] = [filename, 1]
  i += 1

needed_len = len(samples_by_target['marvin']) // 29  # so that dataset contains equal quantity of two classes
i = 0
for key in samples_by_target.keys():
  if key != '_background_noise_' and key != 'marvin':
    for filename in samples_by_target[key][0:needed_len]:
      non_marvin_df.loc[i] = [filename, 0]
      i += 1

In [13]:
marvin_df['class'] = marvin_df['class'].astype(int)
non_marvin_df['class'] = non_marvin_df['class'].astype(int)

In [14]:
marvin_df

,wav_filename,class
0,speech_commands/marvin/682e1687_nohash_1.wav,1
1,speech_commands/marvin/881583a6_nohash_0.wav,1
2,speech_commands/marvin/7014b07e_nohash_2.wav,1
3,speech_commands/marvin/a527cb3c_nohash_1.wav,1
4,speech_commands/marvin/98447c43_nohash_0.wav,1
...,...,...
1741,speech_commands/marvin/300384f0_nohash_0.wav,1
1742,speech_commands/marvin/c0e8f5a1_nohash_1.wav,1
1743,speech_commands/marvin/4f2ab70c_nohash_0.wav,1
1744,speech_commands/marvin/65d14087_nohash_1.wav,1


In [15]:
non_marvin_df

,wav_filename,class
0,speech_commands/yes/ce9410da_nohash_1.wav,0
1,speech_commands/yes/e53139ad_nohash_3.wav,0
2,speech_commands/yes/df1d5024_nohash_2.wav,0
3,speech_commands/yes/333784b7_nohash_1.wav,0
4,speech_commands/yes/16d41d07_nohash_0.wav,0
...,...,...
1735,speech_commands/off/1e4064b8_nohash_1.wav,0
1736,speech_commands/off/64574a99_nohash_0.wav,0
1737,speech_commands/off/ffd2ba2f_nohash_2.wav,0
1738,speech_commands/off/cb2929ce_nohash_3.wav,0


In [16]:
train_marvin = marvin_df.sample(frac=0.9, random_state=21)
test_marvin = marvin_df.drop(train_marvin.index)

train_non_marvin = non_marvin_df.sample(frac=0.9, random_state=21)
test_non_marvin = non_marvin_df.drop(train_non_marvin.index)

train_set = pd.concat([train_marvin, train_non_marvin])
test_set = pd.concat([test_marvin, test_non_marvin])

train_set = train_set.sample(frac=1, random_state=22).reset_index(drop=True)
test_set = test_set.sample(frac=1, random_state=22).reset_index(drop=True)

print("train set size:", len(train_set), "\ntest set size:", len(test_set))

train set size: 3137 
test set size: 349


In [17]:
mel_transform = torchaudio.transforms.MelSpectrogram(
    sample_rate=22050,
    n_fft=1024,
    win_length=1024,
    hop_length=256,
    f_min=0,
    f_max=8000,
    n_mels=n_mels).to(device)

In [18]:
# import IPython

# IPython.display.Audio("speech_commands/marvin/cc592808_nohash_0.wav")  

In [19]:
class MyDataset(Dataset):
    def __init__(self, data):
        super(MyDataset, self).__init__()
        self._data = data

    def __getitem__(self, index: int):
        wav_filename, true_class = self._data.iloc[index]
        # print("file:", wav_filename)
        # path_to_file = path_to_dataset + "wavs/" + wav_filename + ".wav"

        waveform, sample_rate = torchaudio.load(wav_filename)
        waveform = waveform.to(device)
        spectrogram = mel_transform(waveform)
        spectrogram = torch.log(spectrogram + 1e-9)

        return spectrogram, true_class

    def __len__(self):
        return len(self._data)

In [20]:
train_dataset = MyDataset(train_set)
test_dataset = MyDataset(test_set)

In [21]:
# plt.figure(figsize=(20, 5))
# spectrogram, true_class = train_dataset.__getitem__(0)
# print(true_class)
# plt.imshow(spectrogram.cpu().squeeze())
# plt.show()

In [22]:
def augment(waveform):
    freq_transform = torchaudio.transforms.FrequencyMasking(freq_mask_param=freq_mask_param)
    time_transform = torchaudio.transforms.TimeMasking(time_mask_param=time_mask_param)
    return time_transform(freq_transform(waveform))

In [23]:
def collate_sequences(batch, is_test = False):
    specs = []
    labels = []
    specs_len = []
    labels_len = []
    for (log_mel_spec, true_class) in batch:
        # print("true_class:", true_class)
        if not is_test:
            log_mel_spec = augment(log_mel_spec)
        specs.append(log_mel_spec.squeeze(0).transpose(0, 1))
        # specs_len.append(log_mel_spec.shape[-1]//2)
        # print(log_mel_spec.shape)
        # text_in_ints = torch.Tensor(text_transform(text_in_letters))
        labels.append(torch.LongTensor([true_class]))
        # labels_len.append(1)

    # print("labels before cat:", labels)
    specs = nn.utils.rnn.pad_sequence(sequences=specs, batch_first=True).transpose(1, 2)
    # labels = nn.utils.rnn.pad_sequence(sequences=labels, batch_first=True)
    labels = torch.cat(labels)
    # print("labels after cat:", labels)

    return specs, labels  # , specs_len, labels_len

In [24]:
train_dataloader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            collate_fn=collate_sequences,
            num_workers=0
            # pin_memory=True
        )

test_dataloader = DataLoader(
            test_dataset,
            batch_size=batch_size,
            collate_fn=lambda x: collate_sequences(x, True),
            num_workers=0
            # pin_memory=True
        )

In [25]:
print(len(train_dataloader))
print(len(test_dataloader))

99
11


# Model

In [26]:
class CRNNEncoder(nn.Module):
    def __init__(self):
        super(CRNNEncoder, self).__init__()
        self.cnn = nn.Conv1d(
            in_channels=40, out_channels=64, kernel_size=20, padding=20//2
        )
        self.gru = nn.GRU(input_size=64, hidden_size=128)

    def forward(self, x, encoder_hidden):

        outputs = self.cnn(x)

        outputs = torch.transpose(outputs, 0, 2)
        outputs = torch.transpose(outputs, 1, 2)

        if encoder_hidden == None:
            outputs, hidden = self.gru(outputs)
        else:
            outputs, hidden = self.gru(outputs, encoder_hidden)

        outputs = torch.transpose(outputs, 0, 1)

        return outputs, hidden
        
class Attn(nn.Module):
    def __init__(self):
        super(Attn, self).__init__()
        self.main_fully_connected = nn.Linear(in_features=128, out_features=64)
        self.main_softmax = nn.Softmax(dim=-1)
        self.tanh = nn.Tanh()
        self.v = nn.Linear(64, 1)
        self.middle_softmax = nn.Softmax(dim=1)
        self.final_fully_connected = nn.Linear(in_features=128, out_features=2)
        self.final_softmax = nn.Softmax(dim=0)

    def forward(self, hiddens, single_input):

        x = self.main_fully_connected(hiddens)
        x = self.main_softmax(x)
        x = self.tanh(x)

        e = self.v(x)
        a = self.middle_softmax(e)

        outputs = []
        for i in range(hiddens.shape[0]):
          outputs.append(torch.mm(torch.transpose(hiddens[i], 0, 1), a[i]))
        outputs = torch.transpose(torch.cat(outputs, dim=1), 0, 1)

        outputs = self.final_fully_connected(outputs)

        if not single_input:
          outputs = self.final_softmax(outputs)

        return outputs


class KWS(nn.Module):
    def __init__(self):
        super(KWS, self).__init__()
        self.encoder = CRNNEncoder()
        self.attention = Attn()

    def forward(self, x, encoder_hidden = None, single_input = False):

        outputs, hidden = self.encoder(x, encoder_hidden)
        x = self.attention(outputs, single_input)

        return x, hidden
        

# Train-Test pipeline

In [27]:
model=KWS().to(device)
wandb.watch(model)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
criterion = nn.CrossEntropyLoss().to(device)

total_params = sum(p.numel() for p in model.parameters())
print(total_params)

134339


In [28]:
def train(model, train_dataloader, criterion, optimizer, epoch):
    model.train()
    data_len = len(train_dataloader.dataset)
    losses = []
    correct = 0
    total = 0
    fa = 0
    fr = 0
    for batch_idx, (log_mel_specs, classes) in enumerate(train_dataloader):
        log_mel_specs = log_mel_specs.to(device)
        classes = classes.to(device)

        optimizer.zero_grad()

        output, hidden = model(log_mel_specs)  # (batch, time, n_class)
        __, predicted = torch.max(output, dim=1)
        correct += (predicted == classes).sum().item()
        total += classes.shape[0]
        for i in range(classes.shape[0]):
          if (predicted[i] == 1 and classes[i] == 0):
            fa += 1
          elif (predicted[i] == 0 and classes[i] == 1):
            fr += 1
        
        loss = criterion(output, classes)
        loss.backward()
        losses.append(loss)

        optimizer.step()
        if batch_idx == 1 and epoch % 10 == 1:
            print('Train Epoch: {}\tLoss: {:.6f}\tfa/fr: {:.6f}'.format(
                epoch, loss.item(), fa / (fr + 1e-9)))
    return sum(losses) / len(losses), correct, fa, fr + 1e-9, total

In [29]:
def test(model, test_dataloader, criterion, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    fa = 0
    fr = 0
    with torch.no_grad():
        for batch_idx, (log_mel_specs, true_class) in enumerate(test_dataloader):
            log_mel_specs = log_mel_specs.to(device)
            true_class = true_class.to(device)

            output, hidden = model(log_mel_specs)  # (batch, time, n_class)
            __, predicted = torch.max(output, dim=1)
            correct += (predicted == true_class).sum().item()
            total += true_class.shape[0]
            for i in range(true_class.shape[0]):
              if (predicted[i] == 1 and true_class[i] == 0):
                fa += 1
              elif (predicted[i] == 0 and true_class[i] == 1):
                fr += 1
            loss = criterion(output, true_class)
            test_loss += loss.item() / len(test_dataloader)

    print('Test set average loss: {:.4f}\n'.format(test_loss))
    print('Test set fa/fr: {:.4f}\n'.format(fa / (fr + 1e-9)))
    return test_loss, correct, fa, fr + 1e-9, total


In [30]:
# weights_file = wandb.restore('weights.pt')
# model.load_state_dict(torch.load('weights.pt'))

In [31]:
for epoch in range(epochs):
  print("epoch:", epoch)
  train_avg_loss, train_correct, train_fa, train_fr, train_total = train(model, train_dataloader, criterion, optimizer, epoch)
  print("train accuracy:", train_correct / train_total)
  test_avg_loss, test_correct, test_fa, test_fr, test_total = test(model, test_dataloader, criterion, epoch)
  print("test accuracy:", test_correct / test_total)
  wandb.log({"Train loss": train_avg_loss,
             "Test loss": test_avg_loss,
             "Train accuracy": train_correct / train_total,
             "Test accuracy": test_correct / test_total,
             "Train fa": train_fa / train_total,
             "Test fa": test_fa / test_total,
             "Train fr": train_fr / train_total,
             "Test fr": test_fr / test_total})

model = model.to("cpu")
torch.save(model.state_dict(), os.path.join(wandb.run.dir, 'weights.pt'))
wandb.save('weights.h5')
# files.download('model_state_dict.pt')
model = model.to(device)
print("Saved model at last epoch")

epoch: 0
train accuracy: 0.6687918393369461
Test set average loss: 0.6714

Test set fa/fr: 0.7869

test accuracy: 0.6876790830945558
epoch: 1
Train Epoch: 1	Loss: 0.675598	fa/fr: 1.000000
train accuracy: 0.7299968122409946
Test set average loss: 0.6670

Test set fa/fr: 1.0000

test accuracy: 0.7822349570200573
epoch: 2
train accuracy: 0.8042715970672617
Test set average loss: 0.6651

Test set fa/fr: 0.6190

test accuracy: 0.8051575931232091
epoch: 3
train accuracy: 0.8390181702263309
Test set average loss: 0.6647

Test set fa/fr: 0.5294

test accuracy: 0.8510028653295129
epoch: 4
train accuracy: 0.8412496015301243
Test set average loss: 0.6641

Test set fa/fr: 0.8333

test accuracy: 0.8424068767908309
epoch: 5
train accuracy: 0.8645202422696844
Test set average loss: 0.6640

Test set fa/fr: 0.3929

test accuracy: 0.8882521489971347
epoch: 6
train accuracy: 0.8645202422696844
Test set average loss: 0.6639

Test set fa/fr: 0.5333

test accuracy: 0.8681948424068768
epoch: 7
train accuracy

### You can see metrics here: https://wandb.ai/maxthehuman/kws-implementation/runs/1svru9b1?workspace=user-maxthehuman